In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import warnings 

import statsmodels.api as sm
import statsmodels.formula.api as smf          
import statsmodels.tsa.api as smt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

import mylib

# Часть 1

### Считывание данных

In [ ]:
dataFrame = mylib.default_reader('Данные.xlsx')

plt.plot(dataFrame['Value'], label='Values')
plt.legend(fontsize=8)
plt.show()

### Скользящие статистики

In [ ]:
mylib.moving_statistics(dataFrame)

plt.show()

Как видно по графику, временной ряд имеет тренд роста, и не является стационарным. Стандартное отклонение примерно одинаково для простой скользящей статистики.

### Тест Дики-Фуллера

In [ ]:
result = adfuller(dataFrame.Value)

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

Гипотезу о нестационарности нельзя опровергуть, потому что ADF statistic сильно больше критических значений.
Вероятность ошибки при опровержении этой гипотезы - 0.93

### Выводы

Нестационарность легко определяется по графику, и тест Дики-Фуллера подтвержает это.
Наличие тренда очевидно.

# Часть 2

### Тренд, сезональность и остаток аддитивной модели

In [ ]:
mylib.seasonal_decompose_plots(dataFrame, 'additive')

### Тренд, сезональность и остаток мультиптикативной модели

In [ ]:
mylib.seasonal_decompose_plots(dataFrame, 'multiplicative')

### Выводы

Как и было замечено, тренд присутсвует, сезональность также присутсвует. Остаток кажется нестационарным поскольку имеется большая дисперсия в 1960-ом, 1975-ом и 1980-ом году.

# Часть 3

### Проверка на интегрированный ряд порядка k

Используем тест Дики-Фуллера для определения стационарности ряда из разности порядка k с уровнем значимости в 0.01

In [ ]:
mylib.check_periodic(dataFrame.Value, 1, printAns=True, plotting=True)

По тесту Дики-Фуллера и по визуализации видно, что временной ряд - интегрированный порядка 1

### Моделирование ARIMA

Для модели ARIMA необходимо знать параметры <br>
p - порядок компоненты AR <br>
d - порядок интегрированного ряда <br>
q - порядок компонетны MA <br>

q = 1, что установлено в предыдущем отделе <br>
остальные два параметра определяются по графику автокорелляции и частичной автокорелляции

In [ ]:
mylib.autocorrelation_plots(dataFrame, 1)

p = 1 т.к. 1й лаг сильно отличен от 0 <br>
q = 4 т.к. после 4го лага значения достаточно близки к 0 <br>

имеет смысл также рассмотреть модели при p = 0 (потому что 1ый лаг все же на 0.5) и q = 5

In [ ]:
p = 1
d = 1
q = 4

In [ ]:
# frequency information is a bit mysterious
warnings.simplefilter(action='ignore')

model114 = ARIMA(dataFrame.Value, order=(p, d, q)).fit()
model014 = ARIMA(dataFrame.Value, order=(p - 1, d, q)).fit()
model015 = ARIMA(dataFrame.Value, order=(p - 1, d, q + 1)).fit()

In [ ]:
answersData = mylib.default_reader('Ответы.xlsx')

In [ ]:
mylib.predict(model114, answersData)

In [ ]:
mylib.predict(model014, answersData)

In [ ]:
mylib.predict(model015, answersData)

In [ ]:
print(model114.aic)
print(model014.aic)
print(model015.aic)

Следовательно, лучшая модель - ARIMA(0, 1, 5)

# Выводы

Визуально получилось достаточно точно предугадать значения временного ряда. Значения r2_score высокие.
Изначальная модель выбрана неверно

## Постскриптум

ARIMA не работает вообще нигде, почему - неясно. В гугл колабе значения в районе 0. На моей машине - с какого-то момента константа. В интернете такой проблемы не было.